<h1>Pastebin Webscraping</h1>

<h2>Importing libraries</h2>

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
import mysql.connector
import getpass
from datetime import datetime

In [2]:
pswd = getpass.getpass('Password:')

Password:········


*If you ever needed to re-create the table...*

try:
    cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
    cursor = cnx.cursor(buffered=True)
    cursor.execute("CREATE TABLE pastebin (LINK varchar(20), TIME varchar(40))")

except mysql.connector.Error as err:
    print(str(err))
else:
    cursor.close()
    cnx.close()

<h2>Defining the Webscraping Spider</h2>

In [3]:
class MySpider(scrapy.Spider):
    name = "Pastebin"
    
    custom_settings = {
        'DOWNLOAD_DELAY': 15,
    }
    
    
    def __init__(self, *args, **kwargs):
        super(MySpider, self).__init__(*args, **kwargs) 
        self.pswd=[kwargs.get('pswd')] 
    
    def start_requests(self):
        i = 0
        
        seed_url = "https://pastebin.com/trends"
        for i in range(240):
            i+1      
            yield scrapy.Request(url=seed_url, callback=self.parse, dont_filter=True)
        
    def parse(self, response):
        
        def commitSql(page_link_unique):
            try:
                cnx = mysql.connector.connect(host="aintnodatabase.c3hfm1hvxqhx.eu-west-2.rds.amazonaws.com",port=3306,database="pastebin",user="root",password=pswd)
                cursor = cnx.cursor(buffered=True)
                for link in page_link_unique:
                    query = "INSERT INTO pastebin (LINK, TIME) VALUES ('"+link+"','"+str(datetime.now())+"')"
                    cursor.execute(query)
                cnx.commit()
            except mysql.connector.Error as err:
                print(str(err))
            else:
                cursor.close()
                cnx.close()

        p_link = [response.xpath('//ul/li//@href').extract()]
        new_link = []
        [new_link.append(a.encode('ascii','ignore')) for a in p_link[0]]
        commitSql(list(set(new_link) - set(page_link)))
        [page_link.append(a.encode('ascii','ignore')) for a in p_link[0]][-9:]

<h2>Running the Webscraping</h2>

Note, you can't re run the code below in a single session for one reason or another, so you need to restart the kernel between runs.

This code creates a lightweight container for our webspider and then runs it - to be honest understanding this is probably optional unless it breaks.

In [4]:
page_link = []

process = CrawlerProcess()
process.crawl(MySpider, pswd=pswd)
process.start()

2018-05-03 11:08:50 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2018-05-03 11:08:50 [scrapy.utils.log] INFO: Overridden settings: {}
2018-05-03 11:08:50 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-05-03 11:08:50 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddle